In [1]:
import pandas as pd
import os
import string
import nltk
from nltk.corpus import stopwords
import re
from spacy import displacy
import regex
import spacy
import numpy as np
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
nlp = spacy.load('en')

In [25]:
torch.__version__

'1.0.0'

In [2]:
from argparse import Namespace

args = Namespace(
    input_path = './input',
    batch_size = 128,
    fixed_dimension = 500,
    learning_rate = 0.01,
    n_epochs = 100,
    seed = 43
)

In [3]:
from zipfile import ZipFile

with ZipFile(os.path.join(args.input_path, 'yelp.csv.zip')) as zip:
    with zip.open('yelp.csv') as file:
        data = pd.read_csv(file)
        
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [4]:
data = data[['stars', 'text']]
data.head()

,stars,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,5,General Manager Scott Petello is a good egg!!!...


In [5]:
data ['positive'] = data ['stars'] > 3
data ['positive'] = data ['positive'].astype(int)
data.sample(10)

,stars,text,positive
1742,5,I live no where near this place & have gone ou...,1
9551,5,I love this store. The produce is always fres...,1
8240,5,Please excuse this review as I am writing it a...,1
3422,4,"Well, I live down the street from this park at...",1
7160,5,WOW!!! practically half price on everything co...,1
4877,3,Not a bad sub which I find to be something I c...,0
1913,4,BYOB and you can get a flaming boat of sushi?!...,1
9575,3,Mixed reviews from my husband and I:\n\nHis co...,0
6093,5,Teresa S. and I were one of the lucky ones to ...,1
1532,1,"Cold, under done chips. If a Mexican food rest...",0


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

raw_data = data

sns.distplot(data['stars'])

C:\Users\vas\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\stats\stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
C:\Users\vas\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\axes\_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [7]:
y = data ['positive']
data = data ['text']

In [8]:
data.values.ravel()

array(['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!',
       'I have no idea why some people give bad reviews about this place. It goes to show 

In [9]:
spacy.explain('PRON')

'pronoun'

In [10]:
nlp = spacy.load('en')
translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
stops = set(stopwords.words('english'))
bad_pos = set() # set(['SYM','PUNCT','NUM','PART','X','PRON'])

def preprocess(x):
    x = x.lower()
    regex = re.compile("n't")
    x = regex.sub(' not', x)
    x = x.translate(translator)
    regex = re.compile(r'[\+\d]')
    x = regex.sub(' ', x)
    doc = nlp(x)    
    x = [str(word.lemma_).strip() 
         for word in doc if len(str(word).strip()) > 2 and
         word.pos_ not in bad_pos and
         (word.text == 'not' or str(word) not in stops)]
    x = ' '.join(x)
    return x

text = "Vahid 32 Google w324324f 32  is a she this told her? \t \tnn Ms. Zhila \t the better well quitely good the basket run running James \n \r"
docx = nlp(text)
displacy.render(docs=docx, style='ent', jupyter=True)

print('After preprocessing:', preprocess(text))

After preprocessing: vahid google tell zhila good well quitely good basket run run jame


In [11]:
%%time
data = data.apply(lambda x: preprocess(x))

Wall time: 5min 6s


In [12]:
data.shape

(10000,)

In [13]:
data = data.values.ravel()

In [14]:
print ('{} --> {}'.format(data[10], y[10]))
print('{} --> {}'.format(data[15], y[15]))

oldish man own store sweet perhaps sweet cookie ice cream lowdown giant ice cream cookie sandwich super cheap flavor permutation basically endless snickerdoodle cookie cream ice cream marvelous --> 1
worth salad small pizza absolutely not bad service maybe guy grandma die not know want tell really make mad experience order small pizza salad guy could care less take sit look around hmm sign say large pizza large salad wow would nice guy tell leave hungry mad unsatisfied owner teach employee value upsell tell special something small affect customer experience negatively salad severely overprice not back unless desperate --> 0


In [15]:
raw_data = data

In [16]:
# Just for debugging ...
data = raw_data.copy()

data = data.reshape((-1, 1))
data

array([['wife take birthday breakfast excellent weather perfect make sit outside overlook ground absolute pleasure waitress excellent food arrive quickly semi busy saturday morning look like place fill pretty quickly earlier get good favor get bloody mary phenomenal simply good ever pretty sure use ingredient garden blend fresh order amazing everything menu look excellent white truffle scramble egg vegetable skillet tasty delicious come piece griddled bread amazing absolutely make meal complete good toast ever anyway not wait back'],
       ['idea people give bad review place go show please everyone probably grip something fault many people like case friend arrive past sunday pretty crowded think sunday evening think would wait forever get seat say seat girl come back seat someone else seat waiter come get drink order everyone pleasant host seat waiter server price good well place order decide want share baked spaghetti calzone small beef pizza try calzone huge get small one personal g

In [17]:
doc = {}
vocab_size = 1
def string_to_ints(my_input):
    x = str(my_input)
    global vocab_size
    words = x.split()
    res = []
    for word in words:
        if word not in doc:
            doc[word] = vocab_size
            vocab_size += 1
        res.append(doc[word])
    return np.asarray(res)
def lookup_word(word):
    if word in doc:
        return doc [word]
    return 0

In [18]:
temp = []
for i in range(data.shape[0]):
    temp.append(string_to_ints(data [i]))

In [19]:
data = np.asarray(temp)

In [20]:
def make_fixed_columns(row, fixed_size):
    if len(row) < fixed_size:
        return np.concatenate((np.zeros(fixed_size - len(row)), row), axis = None).tolist()
    if len(row) > fixed_size:
        return row[-fixed_size:].tolist()
    return row.tolist()

In [21]:
for i in range (data.shape[0]):
    data [i] = make_fixed_columns(data[i], args.fixed_dimension)
data

array([list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [22]:
len(np.concatenate(data).ravel())
data = np.concatenate(data).reshape(-1, args.fixed_dimension).astype(np.int16)

In [23]:
data.shape

(10000, 500)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

split_sz = 0.9
sz = int(split_sz * data.shape[0])
X_train, y_train = data [0 : sz], y [0 : sz]
X_test, y_test = data [sz :], y [sz :]
data.shape, sz, X_train.shape, X_test.shape

((10000, 500), 9000, (9000, 500), (1000, 500))

In [26]:
from sklearn.preprocessing import normalize
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
def generate_minibatch (X, y): # X and y are numpy matrixes 
    X, y = shuffle(X, y)
    for i in range (0, X.shape [0], args.batch_size): #batch_size is equal to 128
        X_mini = X [i:i + args.batch_size]
        y_mini = y [i:i + args.batch_size]
        
        y_mini = y_mini.reshape(-1, 1)
        
        X_mini = torch.LongTensor(X_mini)
        y_mini = torch.FloatTensor(y_mini)
        
        y_mini = y_mini.view(-1, 1)
        
        yield X_mini, y_mini

In [27]:
class reviewClassifier(nn.Module):
    def __init__(self):
        super(reviewClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size + 1, 100)
        self.fc1 = nn.Linear(in_features=args.fixed_dimension * 100, out_features=64) # fixed dimension is equal to 20
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.fc3 = nn.Linear(in_features=32, out_features=1)
        #self.fc4 = nn.Linear(in_features=64, out_features=32)
        #self.fc5 = nn.Linear(in_features=32, out_features=1)
    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.dropout(x, 0.5)
        x = self.fc3(x)
        #x = F.relu(self.fc4(x))
        #x = F.tanh(self.fc5(x))
        y = torch.sigmoid(x)
        return y

In [28]:
%%time
classifier = reviewClassifier()

loss_func = nn.BCELoss()

optimizer = optim.Adam(classifier.parameters(), lr=0.001)

print(np.mean(X_train))


for epoch in range(100):
    total_loss = 0 
    for X_mini, y_mini in generate_minibatch(X_train, y_train):
        
        classifier.zero_grad()
        
        y_pred = classifier.forward(x=X_mini.long())
        
        loss = loss_func(y_pred, y_mini)
        total_loss = torch.add(total_loss, loss.data)
        
        loss.backward()
        optimizer.step()
        
        
    if epoch % 10 == 0:
        print('Loss = ', total_loss)
        
for idx , param in enumerate (list(classifier.parameters())):
    print(' >> ', idx , param.grad)
print('Finished!')

274.351462
Loss =  tensor(64.3256)
Loss =  tensor(4.7997)
Loss =  tensor(0.4723)
Loss =  tensor(0.2018)
Loss =  tensor(0.8580)
Loss =  tensor(0.1971)
Loss =  tensor(0.0834)
Loss =  tensor(0.0706)
Loss =  tensor(35.1574)
Loss =  tensor(0.0317)
 >>  0 tensor([[-1.2208e-03,  7.4828e-04, -1.0264e-03,  ...,  1.2696e-03,
         -1.4627e-03,  1.5154e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.8802e-07,  3.5613e-07,  2.3388e-07,  ...,  4.8480e-07,
          2.7101e-07,  3.9217e-07],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])
 >>  1 tensor([[-5.3945e-05, -3.0929e-06, -5.0842e-05,  ...,  2.2913e-05,
          8.4909e-05,  6.2447e-05],
        

In [29]:
X_train[20:30]

array([[  0,   0,   0, ..., 359, 688, 764],
       [  0,   0,   0, ...,  25, 104, 766],
       [  0,   0,   0, ...,  72,  93, 772],
       ...,
       [  0,   0,   0, ..., 878, 879, 880],
       [  0,   0,   0, ..., 905, 328, 906],
       [  0,   0,   0, ..., 916, 917, 918]], dtype=int16)

In [30]:
np.asarray(y_train [20:30].tolist())

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [31]:
X_test.shape

(1000, 500)

In [35]:
test = torch.LongTensor(X_test).long()

y_pred = classifier(x=test)
y_pred = y_pred.view(-1, 1)
y_pred = y_pred.detach().numpy()
y_pred = np.where(y_pred < 0.5, 0, 1)
y_test = y_test.reshape(-1, 1)
print('Accuracy = {:.2%}'.format(np.mean(y_pred == y_test)))

Accuracy = 78.80%


In [36]:
text = 'I would try this food not any more' 
# I know this sentence is grammatically wrong but I just wanted to know how powerful our model is. 
#It properly realizes that the comment is negative.
print(preprocess(text))

my_test = make_fixed_columns (string_to_ints([preprocess(text)]), args.fixed_dimension)
print(my_test, vocab_size)
my_test = np.asarray(my_test)
my_test = my_test.reshape(1, -1)
test = torch.LongTensor(my_test).long()
print(test.shape)
y_pred = classifier.forward(x=test)
print('{:.10%}'.format(y_pred.detach().numpy()[0][0]))

would try food not
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 